In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

***크롬드라이버 실행(자기 크롬 버전에 맞게 설치후 test.ipynb와 같은 경로에 두기)***

In [ ]:
driver = webdriver.Chrome('chromedriver')
url_list = ["36528615", "31317229", "1269259731", "1779562085", "33040151", "1675801137", "1384361907", "1372982703", "1681990192", "1971062401"]


***클릭할 버튼이 없을때까지 스크롤 내림 (일단 임시로 100지정)***

In [ ]:
from selenium.webdriver.common.by import By


In [ ]:
import time
import pandas as pd
userlink = pd.DataFrame()
for _url in url_list[0]:
    URL = f"https://m.place.naver.com/restaurant/{_url}/review/visitor"  
    driver.get(URL)
    time.sleep(2.5)
    for _ in range(100):
        driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
        time.sleep(0.5)
        try: driver.find_element(By.CLASS_NAME, 'fvwqf').click()
        except: pass
        time.sleep(0.1)
    
    link_list = driver.find_elements(By.CLASS_NAME, "Hazns")
    link_list = [link.get_attribute("href") for link in link_list]
    user_list = driver.find_elements(By.CLASS_NAME, "sBWyy")
    user_list = [user.text for user in user_list]
    review_time_list = driver.find_elements(By.CLASS_NAME, "sb8UA")
    review_time_list = [reviews.text for reviews in review_time_list]
    length = min(len(user_list), len(link_list), len(review_time_list))
    userlink2 = pd.DataFrame({'user' : user_list[:length], 'link' : link_list[:length], 'review_time' : review_time_list[:length]}, dtype = str)
    userlink = pd.concat([userlink, userlink2], axis = 0, sort=False)

In [ ]:
# iframe 아이디를 반환해줌 
# iframes = driver.find_elements(By.CSS_SELECTOR, 'iframe')
# for iframe in iframes:
#     print(iframe.get_attribute('id'))


In [ ]:
# driver.switch_to.parent_frame() # 아래 코드를 이용해서 iframe갔을때 빠져나오기 위한 코드 (현재는 의미없음)

**식당 리스트 뽑기위한 코드**

In [ ]:
import time
import pandas as pd
driver = webdriver.Chrome('chromedriver')
URL = "https://map.naver.com/v5/search/%EC%84%9C%EC%B4%88%EA%B5%AC%20%EC%9D%8C%EC%8B%9D%EC%A0%90?c=14137477.8183780,4506726.1284877,12,0,0,0,dh"  
driver.get(URL)

In [ ]:
driver.switch_to.frame('searchIframe') #iframe 으로 이동하기 위한 코드
restaurants_list = pd.DataFrame()

In [69]:
from tqdm import tqdm
epoch = 6 # 6이 최대인듯
for _ in tqdm(range(epoch)):
    click_list = driver.find_elements(By.CLASS_NAME, "tzwk0")
    click_list[0].click() #아무거나 클릭을 해야 end키가 먹혀서 임의로 아무 클래스 클릭한거임
    for _ in range(10):
        driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END) # end키를 눌러서 아래까지 로딩
    time.sleep(2)
    click_list = driver.find_elements(By.CLASS_NAME, "tzwk0") # 식당 리스트
    name_list = driver.find_elements(By.CLASS_NAME, "TYaxT") # 식당 이름 리스트
    name_list = [user.text for user in name_list]
    tag_list = driver.find_elements(By.CLASS_NAME, "KCMnt") # 식당 종류 리스트
    tag_list = [user.text for user in tag_list]
    url_list = []
    for restaurant_link in click_list:
        restaurant_link.click()
        url_list.append(driver.current_url)
        time.sleep(0.5)
    restaurants_list2 = pd.DataFrame({'restaurant' : name_list, 'tag' : tag_list, 'url' : url_list}, dtype = str)
    restaurants_list = pd.concat([restaurants_list, restaurants_list2], axis = 0, sort=False)
    driver.find_elements(By.CLASS_NAME, "yUtES")[1].click() #다음페이지 클릭
    time.sleep(2)

100%|██████████| 6/6 [01:10<00:00, 11.80s/it]


In [71]:
restaurants_list.to_csv('restaurant.csv', index=False)